In [5]:
import numpy as np
import cv2
import pandas as pd
import imageio as im
from PIL import Image
import os
import dlib
from imutils.face_utils.helpers import rect_to_bb

In [3]:
ls

glass/  glasses/  glass.ipynb  labels.npy  w_glass/  without_glasses/


In [7]:
## Cropping the faces from images

# detector = dlib.get_frontal_face_detector()
# dest = 'glass/'
# img_dir = ['glasses/']
# for im_dir in img_dir:
#     images = os.listdir(im_dir)
#     for i,image in enumerate(images):
#         img = cv2.imread(im_dir + image)
#         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         rects = detector(gray, 1) 
#         for j,rect in enumerate(rects):
#             (x, y, w, h) = rect_to_bb(rect)
# #             cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,255),1)
#             sub_face = gray[y:y+h, x:x+w]
#             cv2.imwrite(dest+str(i)+'_'+str(j)+'.jpg',sub_face)

In [16]:
all_images = []
labels = []
img_dir = ['glass/','w_glass/']
for im_dir in img_dir:
    images = os.listdir(im_dir)
    for image in images:
        img = cv2.imread(im_dir + image)
        img = cv2.resize(img, (36, 36)) # need to resize the image into common size
        all_images.append(img)
        labels.append(img_dir.index(im_dir))

In [48]:
img = cv2.imread('glass/101_0.jpg')
img.shape

(75, 75, 3)

In [19]:
all_images[0].shape

(36, 36, 3)

In [20]:
data = np.array(all_images)
data = data.astype('float32')
data /= 255.0
len(data)

592

In [21]:
# np.save(open('glass.npy', 'wb'), data)
# np.save(open('labels.npy', 'wb'), y_train)

In [22]:
from sklearn.preprocessing import OneHotEncoder
labels = np.asarray(labels)
labels = labels.reshape(-1, 1)
ohe_y=OneHotEncoder()
Y=ohe_y.fit_transform(labels)
Y.shape

/home/compindia/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(592, 2)

In [24]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
x,y = shuffle(data,Y, random_state=42)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [29]:
import keras
from keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Model, Sequential
from keras.optimizers import Adam

classifier=Sequential()

classifier.add(Convolution2D(32,3,3,activation='relu',input_shape=(36, 36, 3)))

classifier.add(Convolution2D(64,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))


classifier.add(Convolution2D(128,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Convolution2D(256,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))


classifier.add(Flatten())

classifier.add(Dense(256,activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(2,activation='softmax'))

classifier.compile(Adam(lr=0.0001, decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])

/home/compindia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(36, 36, 3...)`
  
/home/compindia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
/home/compindia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  from ipykernel import kernelapp as app
/home/compindia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`


In [30]:
classifier.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=1, shuffle=True, verbose=1 )

Train on 473 samples, validate on 119 samples
Epoch 1/25
473/473 [==============================] - 13s 27ms/step - loss: 0.6988 - acc: 0.5011 - val_loss: 0.6704 - val_acc: 0.5126
Epoch 2/25
473/473 [==============================] - 12s 25ms/step - loss: 0.5737 - acc: 0.7273 - val_loss: 0.4474 - val_acc: 0.7731
Epoch 3/25
473/473 [==============================] - 12s 25ms/step - loss: 0.3791 - acc: 0.8520 - val_loss: 0.3157 - val_acc: 0.8655
Epoch 4/25
473/473 [==============================] - 12s 25ms/step - loss: 0.3286 - acc: 0.8647 - val_loss: 0.2757 - val_acc: 0.8824
Epoch 5/25
473/473 [==============================] - 12s 25ms/step - loss: 0.2948 - acc: 0.8837 - val_loss: 0.2512 - val_acc: 0.8992
Epoch 6/25
473/473 [==============================] - 12s 25ms/step - loss: 0.2245 - acc: 0.9154 - val_loss: 0.2257 - val_acc: 0.9076
Epoch 7/25
473/473 [==============================] - 12s 25ms/step - loss: 0.2230 - acc: 0.9070 - val_loss: 0.2497 - val_acc: 0.8824
Epoch 8/25
473/4

In [31]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 34, 34, 32)        896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 5, 5, 256)         295168    
__________

In [32]:
all_images = []
labels = []
img_dir = ['glass/','w_glass/']
for im_dir in img_dir:
    images = os.listdir(im_dir)
    for image in images:
        img = cv2.imread(im_dir + image)
        img = cv2.resize(img, (48, 48)) # need to resize the image into common size
        all_images.append(img)
        labels.append(img_dir.index(im_dir))

In [33]:
all_images[0].shape

(48, 48, 3)

In [34]:
data = np.array(all_images)
data = data.astype('float32')
data /= 255.0
len(data)

592

In [35]:
# np.save(open('glass.npy', 'wb'), data)
# np.save(open('labels.npy', 'wb'), y_train)

from sklearn.preprocessing import OneHotEncoder
labels = np.asarray(labels)
labels = labels.reshape(-1, 1)
ohe_y=OneHotEncoder()
Y=ohe_y.fit_transform(labels)
Y.shape

/home/compindia/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(592, 2)

In [36]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
x,y = shuffle(data,Y, random_state=42)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

import keras
from keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Model, Sequential
from keras.optimizers import Adam

classifier=Sequential()

classifier.add(Convolution2D(32,3,3,activation='relu',input_shape=(48, 48, 3)))

classifier.add(Convolution2D(64,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))


classifier.add(Convolution2D(128,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Convolution2D(256,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))


classifier.add(Flatten())

classifier.add(Dense(256,activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(2,activation='softmax'))

classifier.compile(Adam(lr=0.0001, decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])


classifier.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=1, shuffle=True, verbose=1 )

/home/compindia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(48, 48, 3...)`
  
/home/compindia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  app.launch_new_instance()
/home/compindia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
/home/compindia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`


Train on 473 samples, validate on 119 samples
Epoch 1/25
473/473 [==============================] - 25s 52ms/step - loss: 0.6941 - acc: 0.5455 - val_loss: 0.6039 - val_acc: 0.8067
Epoch 2/25
473/473 [==============================] - 26s 55ms/step - loss: 0.4532 - acc: 0.7907 - val_loss: 0.3476 - val_acc: 0.8571
Epoch 3/25
473/473 [==============================] - 23s 49ms/step - loss: 0.3223 - acc: 0.8795 - val_loss: 0.2634 - val_acc: 0.8992
Epoch 4/25
473/473 [==============================] - 24s 51ms/step - loss: 0.2488 - acc: 0.9091 - val_loss: 0.2190 - val_acc: 0.9244
Epoch 5/25
473/473 [==============================] - 22s 47ms/step - loss: 0.2520 - acc: 0.9049 - val_loss: 0.2641 - val_acc: 0.8908
Epoch 6/25
473/473 [==============================] - 22s 47ms/step - loss: 0.2158 - acc: 0.9175 - val_loss: 0.2008 - val_acc: 0.9244
Epoch 7/25
473/473 [==============================] - 22s 46ms/step - loss: 0.2156 - acc: 0.9197 - val_loss: 0.2001 - val_acc: 0.9076
Epoch 8/25
473/4

In [ ]:
import imutils

In [74]:
src = 'test/'
for images in os.listdir('test/'):
    image = cv2.imread(src+images)
    image = imutils.resize(image, width=700)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)   # detect faces in the grayscale image
    for i,rect in enumerate(rects):
        print(rect)
        (x, y, w, h) = rect_to_bb(rect)
        cv2.rectangle(image,(x,y),(x+w,y+h),(255,255,255),1)
        cv2.putText(image,"FACE #{}".format(i+1),(x-10,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
        x = max(0,rect.left())
        y = max(0,rect.top())
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        cv2.imwrite('test/test.jpg',face)
        face1 = cv2.imread('test/test.jpg')
        face = face1.reshape((1,48, 48, 3))
        result = classifier.predict(face)[0]
        res = 'NO' if result[1] > result[0] else 'YES'
#         print(result)
        cv2.putText(image,"Glasses :{}".format(res),(x+10,y+10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
        cv2.imshow('image',image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

[(184, 64) (339, 219)]
(48, 48)
(1, 48, 48, 3)
[1. 0.]
[(469, 222) (655, 407)]
(48, 48)
(1, 48, 48, 3)
[1. 0.]
[(15, 242) (201, 428)]
(48, 48)
(1, 48, 48, 3)
[1. 0.]
[(43, 142) (266, 365)]
(48, 48)
(1, 48, 48, 3)
[0. 1.]
[(464, 93) (687, 316)]
(48, 48)
(1, 48, 48, 3)
[1. 0.]
[(96, 82) (225, 211)]
(48, 48)
(1, 48, 48, 3)
[0. 1.]
[(442, 81) (597, 236)]
(48, 48)
(1, 48, 48, 3)
[0. 1.]
[(27, 241) (348, 562)]
(48, 48)
(1, 48, 48, 3)
[0. 1.]
[(332, 204) (718, 590)]
(48, 48)
(1, 48, 48, 3)
[1. 0.]
[(110, 110) (664, 665)]
(48, 48)
(1, 48, 48, 3)
[1. 0.]
[(27, 134) (348, 455)]
(48, 48)
(1, 48, 48, 3)
[0. 1.]
[(332, 119) (718, 504)]
(48, 48)
(1, 48, 48, 3)
[1. 0.]
[(439, 142) (662, 365)]
(48, 48)
(1, 48, 48, 3)
[1. 0.]
[(92, 118) (315, 341)]
(48, 48)
(1, 48, 48, 3)
[1.0702001e-20 1.0000000e+00]
[(386, 57) (572, 242)]
(48, 48)
(1, 48, 48, 3)
[0. 1.]
[(97, 57) (283, 242)]
(48, 48)
(1, 48, 48, 3)
[1. 0.]
[(442, 305) (597, 460)]
(48, 48)
(1, 48, 48, 3)
[0. 1.]
[(139, 268) (268, 397)]
(48, 48)
(1, 48

In [75]:
classifier.save('glass_model.h5')

In [18]:
# import keras
# target_size = 224
# from keras.applications.mobilenetv2 import MobileNetV2
# from keras.layers import Dense, Input, Dropout
# from keras.models import Model

# def build_model( ):
#     input_tensor = Input(shape=(target_size, target_size, 3))
#     base_model = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_tensor,
#         input_shape=(target_size, target_size, 3), pooling='avg')

#     for layer in base_model.layers:
#         layer.trainable = True  # trainable has to be false in order to freeze the layers
#     op = Dense(1, activation = 'softmax')(base_model.output)

#     output_tensor = Dense(1, activation='softmax')(op)

#     model = Model(inputs=input_tensor, outputs=output_tensor)

#     return model

# from keras.optimizers import Adam
# model = build_model()
# model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# model.fit(X_train, y_train, epochs = 5, verbose = 1,batch_size=1 ,validation_data=(X_test,y_test))